# Convert the Raw Data into a Dictionary
This notebook is designed to read in the master file and create an organized dictionary that will be used in the following analysis. Again, no need to re-run this cell since the full dictionary is housed in "full_plays_dictionary.pckl"

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pickle
from tqdm import tqdm
import pandas as pd

d = pd.read_csv('master_chrono.csv')


In [ ]:
#Loop through each gameID and grab the plays within it -> Code block has a run time of ~3 hours to complete.
pret = {'rush':{},'pass':{}}

gameIDs = np.unique(d.gameId.values)
for val in tqdm(gameIDs,total=len(gameIDs)):

    dgame = d.loc[np.where(d.gameId==val)[0]] #Dataframe of the gameID
    dgame.reset_index(inplace=True,drop=True)

    playIDs = np.unique(dgame.playId.values)

    #Isolate the play
    for play in playIDs:

        dplay = dgame.loc[np.where(dgame.playId==play)[0]]
        dplay.reset_index(inplace=True,drop=True)

        #Log the gameId and playId
        if str(val)+'-'+str(play) not in pret:
            if dplay.rush.values[0] == 1:
                pret['rush'][str(val)+'-'+str(play)] = {}
            else:
                pret['pass'][str(val)+'-'+str(play)] = {}

        #Parse through each frame within the play
        frames = np.unique(dplay.frameId.values)
        for frame in frames:

            dframe = dplay.loc[np.where(dplay.frameId==frame)[0]]
            dframe.reset_index(inplace=True,drop=True)

            #Isolate all players within this frame
            nflIDs = np.unique(dframe.nflId.values)
            for player in nflIDs:

                dplayer = dframe.loc[np.where(dframe.nflId==player)[0]]
                dplayer.reset_index(inplace=True,drop=True)

                #Log the player on this frame
                if dplayer.team.values[0]==dplayer.homeTeamAbbr.values[0]:
                    home_bool = 'home'
                elif dplayer.team.values[0]==dplayer.visitorTeamAbbr.values[0]:
                    home_bool = 'away'
                else:
                    home_bool = dplayer.team.values[0] #Should be 'football'

                alist = dplayer[['x','y','s','a','dis','o','dir','jerseyNumber','playDirection','officialPosition','epa']].values[0].tolist()
                alist = np.append(alist,home_bool).tolist()
                if dplay.rush.values[0] == 1:
                    if str(frame) not in pret['rush'][str(val)+'-'+str(play)]:
                        pret['rush'][str(val)+'-'+str(play)][str(frame)] = {}
                    pret['rush'][str(val)+'-'+str(play)][str(frame)][str(dplayer.nflId.values[0])] = alist
                else:
                    if str(frame) not in pret['pass'][str(val)+'-'+str(play)]:
                        pret['pass'][str(val)+'-'+str(play)][str(frame)] = {}
                    pret['pass'][str(val)+'-'+str(play)][str(frame)][str(dplayer.nflId.values[0])] = alist


In [ ]:
#An example of how this dictionary stores the data
pret['pass']['2021090900-97']['1']['42377.0']

In [ ]:
#Save this dictionary to pickle
f = open('full_plays_dictionary.pckl', 'wb')
pickle.dump(pret, f)
f.close()


In [1]:
pret['pass']['2021091913-2775']

NameError: name 'pret' is not defined